# Gradient & Sigmoid Windowing

I've been exploring a bunch of different ways to window the DICOM images and I thought I'd share a few ideas and results.

Huge thanks to [David Tang](https://www.kaggle.com/dcstang/see-like-a-radiologist-with-systematic-windowing), [Marco](https://www.kaggle.com/marcovasquez/basic-eda-data-visualization), [Nanashi](https://www.kaggle.com/jesucristo/rsna-introduction-eda-models), and [Richard McKinley](https://www.kaggle.com/omission/eda-view-dicom-images-with-correct-windowing) and for their amazing Kernels that I totally borrowed code and ideas from.

**Contents**
* [No Windowing](#1)
* [Brain Windowing](#2)
* [Metadata Windowing](#3)
* [One Window, Three Channels](#4)
* [Gradient Windowing](#5)
* [Brain + Subdural + Bone Windowing](#6)
* [Exclusive Windowing](#7)
* [Gradient (Brain + Subdural + Bone) Windowing](#8)
* [Sigmoid Windowing](#9)
* [Sigmoid (Brain + Subdural + Bone) Windowing](#10)
* [Sigmoid Gradient (Brain + Subdural + Bone) Windowing](#11)
* [Acknowledgements](#12)

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import pydicom
import os

TRAIN_IMG_PATH = "../input/rsna-intracranial-hemorrhage-detection/stage_1_train_images/"
TEST_IMG_PATH = "../input/rsna-intracranial-hemorrhage-detection/stage_1_test_images/"
BASE_PATH = '/kaggle/input/rsna-intracranial-hemorrhage-detection/'
TRAIN_DIR = 'stage_1_train_images/'

train = pd.read_csv("../input/rsna-intracranial-hemorrhage-detection/stage_1_train.csv")
train_images = os.listdir("../input/rsna-intracranial-hemorrhage-detection/stage_1_train_images/")

train['filename'] = train['ID'].apply(lambda st: "ID_" + st.split('_')[1] + ".dcm")
train['type'] = train['ID'].apply(lambda st: st.split('_')[2])
train = train[['Label', 'filename', 'type']].drop_duplicates().pivot(index='filename', columns='type', values='Label').reset_index()

hem_types = ['epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural']


def load_random_images():
    image_names = [list(train[train[h_type] == 1].sample(1)['filename'])[0] for h_type in hem_types]
    image_names += list(train[train['any'] == 0].sample(5)['filename'])
    return [pydicom.read_file(os.path.join(TRAIN_IMG_PATH, img_name)) for img_name in image_names]


def view_images(images):
    width = 5
    height = 2
    fig, axs = plt.subplots(height, width, figsize=(15,5))
    
    for im in range(0, height * width):
        image = images[im]
        i = im // width
        j = im % width
        axs[i,j].imshow(image, cmap=plt.cm.bone) 
        axs[i,j].axis('off')
        title = hem_types[im] if im < len(hem_types) else 'normal'
        axs[i,j].set_title(title)

    plt.show()
    

def get_first_of_dicom_field_as_int(x):
    #get x[0] as in int is x is a 'pydicom.multival.MultiValue', otherwise get int(x)
    if type(x) == pydicom.multival.MultiValue:
        return int(x[0])
    else:
        return int(x)

    
def get_windowing(data):
    dicom_fields = [data[('0028','1050')].value, #window center
                    data[('0028','1051')].value, #window width
                    data[('0028','1052')].value, #intercept
                    data[('0028','1053')].value] #slope
    return [get_first_of_dicom_field_as_int(x) for x in dicom_fields]

print('Loaded packages and setup utility functions')

Let's load a some images. We'll randomly pick one positive example from each class and 5 negative (normal) examples. 

In [ ]:
imgs = load_random_images()

In [ ]:
print('DICOM Pixel Data Shape: ', imgs[0].pixel_array.shape)

In [ ]:
plt.title('Distribution of DICOM Pixel Values')
ax = plt.hist(np.array([img.pixel_array for img in imgs]).flatten(), bins=50, color='c')

We can see the DICOM data is 2-dimensional and has a range of values much wider than the typical png or jpg image. Let's remind ourselves what the scans look like if we include the full range of values... 

### No Windowing <a></a>

In [ ]:
view_images([img.pixel_array for img in imgs])

These don't look very useful for detecting hemorrhages.

[David Tang's Kernel](https://www.kaggle.com/dcstang/see-like-a-radiologist-with-systematic-windowing) and [Richard McKinley's Kernel](https://www.kaggle.com/omission/eda-view-dicom-images-with-correct-windowing) both do a great job at explaining windowing and how it's used by radiologists. (Make sure to check them out!)

The way I've been thinking about it, is **how do we transform our 2D scan data into 3D image data in a way that makes it easy for our model to detect hemorrhages?**


### Brain Windowing <a></a>
Let's check out the range of values that corresponds with brain matter. We'll clip everything outside that range so that there's more contrast in the brain-range.

In [ ]:
def brain_window(img):
    window_min = 0
    window_max = 80
    _, _, intercept, slope = get_windowing(img)
    img = img.pixel_array
    img = img * slope + intercept
    img[img < window_min] = window_min
    img[img > window_max] = window_max
    img = (img - np.min(img)) / (np.max(img) - np.min(img))
    return img

view_images([brain_window(img) for img in imgs])

### Metadata Windowing <a></a>
The DICOM images come with metadata specifying a window center and width. We could also use these values instead of the fixed range from above.

In [ ]:
def metadata_window(img, print_ranges=True):
    # Get data from dcm
    window_center, window_width, intercept, slope = get_windowing(img)
    img = img.pixel_array
    
    # Window based on dcm metadata
    img = img * slope + intercept
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    if print_ranges:
        print(img_min, img_max)
    img[img < img_min] = img_min
    img[img > img_max] = img_max
    
    # Normalize
    img = (img - img_min) / (img_max - img_min)
    return img
    

print('Metadata Window Ranges:')
view_images([metadata_window(img) for img in imgs])

Looks like the metadata ranges are somewhat similar to the brain-range we initially used.

### One Window, Three Channels <a></a>
Since we'd like to eventually export the scans as png files, we have 3 channels (R,G,B) to work with. If we're only going to use one window setting, we can try to improve the contrast by spreading it out across all 3 channels.

In [ ]:
def all_channels_window(img):
    grey_img = brain_window(img) * 3.0
    all_chan_img = np.zeros((grey_img.shape[0], grey_img.shape[1], 3))
    all_chan_img[:, :, 2] = np.clip(grey_img, 0.0, 1.0)
    all_chan_img[:, :, 0] = np.clip(grey_img - 1.0, 0.0, 1.0)
    all_chan_img[:, :, 1] = np.clip(grey_img - 2.0, 0.0, 1.0)
    return all_chan_img
    

view_images([all_channels_window(img) for img in imgs])

### Gradient Windowing <a></a>
We can spread our a single window across channels a different way, by mapping the pixel values to a gradient.

In [ ]:
def map_to_gradient(grey_img):
    rainbow_img = np.zeros((grey_img.shape[0], grey_img.shape[1], 3))
    rainbow_img[:, :, 0] = np.clip(4 * grey_img - 2, 0, 1.0) * (grey_img > 0) * (grey_img <= 1.0)
    rainbow_img[:, :, 1] =  np.clip(4 * grey_img * (grey_img <=0.75), 0,1) + np.clip((-4*grey_img + 4) * (grey_img > 0.75), 0, 1)
    rainbow_img[:, :, 2] = np.clip(-4 * grey_img + 2, 0, 1.0) * (grey_img > 0) * (grey_img <= 1.0)
    return rainbow_img

def rainbow_window(img):
    grey_img = brain_window(img)
    return map_to_gradient(grey_img)

view_images([rainbow_window(img) for img in imgs])

### Brain + Subdural + Bone Windowing <a></a>
As David points out in his Kernel, different hemmorhages become more obvious at different window settings.. We can include more than one window in our training images by storing a different window in each channel.

In [ ]:
def window_image(img, window_center, window_width):
    _, _, intercept, slope = get_windowing(img)
    img = img.pixel_array * slope + intercept
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    img[img < img_min] = img_min
    img[img > img_max] = img_max
    img = (img - np.min(img)) / (np.max(img) - np.min(img))
    return img


def bsb_window(img):
    brain_img = window_image(img, 40, 80)
    subdural_img = window_image(img, 80, 200)
    bone_img = window_image(img, 600, 2000)
    
    bsb_img = np.zeros((brain_img.shape[0], brain_img.shape[1], 3))
    bsb_img[:, :, 0] = brain_img
    bsb_img[:, :, 1] = subdural_img
    bsb_img[:, :, 2] = bone_img
    return bsb_img

view_images([bsb_window(img) for img in imgs])

### Exclusive Windowing <a></a>
Same idea as above, but removing values outside the window range by setting them to 0.

In [ ]:
def window_image_bottom(img, window_center, window_width):
    _, _, intercept, slope = get_windowing(img)
    img = img.pixel_array * slope + intercept
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    img[img < img_min] = img_min
    img[img > img_max] = img_min
    img = (img - np.min(img)) / (np.max(img) - np.min(img))
    return img


def bsb_window(img):
    brain_img = window_image_bottom(img, 40, 80)
    subdural_img = window_image_bottom(img, 80, 200)
    bone_img = window_image_bottom(img, 600, 2000)
    
    bsb_img = np.zeros((brain_img.shape[0], brain_img.shape[1], 3))
    bsb_img[:, :, 0] = brain_img
    bsb_img[:, :, 1] = subdural_img
    bsb_img[:, :, 2] = bone_img
    return bsb_img

view_images([bsb_window(img) for img in imgs])

### Gradient (Brain + Subdural + Bone) Windowing <a></a>
We can combine a few previous ideas by averaging 3 different window settings and then mapping the results to a gradient. 

In [ ]:
def rainbow_bsb_window(img):
    brain_img = window_image(img, 40, 80)
    subdural_img = window_image(img, 80, 200)
    bone_img = window_image(img, 600, 2000)
    combo = (brain_img*0.3 + subdural_img*0.5 + bone_img*0.2)
    return map_to_gradient(combo)

view_images([rainbow_bsb_window(img) for img in imgs])

### Sigmoid Windowing <a></a>
Instead of simply clipping values outside of our window, we can use a sigmoid function to increase the variance near the middle of the window, while limiting the variance at the extremes. (I didn't come up with this, I believe it's a pretty common way to window.)

This looks like it does a better job creating contrast near the center of the window and we aren't losing any data like we do when we clip to a min/max.

In [ ]:
def sigmoid_window(img, window_center, window_width, U=1.0, eps=(1.0 / 255.0)):
    _, _, intercept, slope = get_windowing(img)
    img = img.pixel_array * slope + intercept
    ue = np.log((U / eps) - 1.0)
    W = (2 / window_width) * ue
    b = ((-2 * window_center) / window_width) * ue
    z = W * img + b
    img = U / (1 + np.power(np.e, -1.0 * z))
    img = (img - np.min(img)) / (np.max(img) - np.min(img))
    return img

def sigmoid_brain_window(img):
    return sigmoid_window(img, 40, 80)

view_images([sigmoid_brain_window(img) for img in imgs])

### Sigmoid (Brain + Subdural + Bone) Windowing <a></a>
Again, combining two previous ideas.

In [ ]:
def sigmoid_bsb_window(img):
    brain_img = sigmoid_window(img, 40, 80)
    subdural_img = sigmoid_window(img, 80, 200)
    bone_img = sigmoid_window(img, 600, 2000)
    
    bsb_img = np.zeros((brain_img.shape[0], brain_img.shape[1], 3))
    bsb_img[:, :, 0] = brain_img
    bsb_img[:, :, 1] = subdural_img
    bsb_img[:, :, 2] = bone_img
    return bsb_img

view_images([sigmoid_bsb_window(img) for img in imgs])

### Sigmoid Gradient (Brain + Subdural + Bone) Windowing <a></a>
And finally, putting it all together.

In [ ]:
def map_to_gradient_sig(grey_img):
    rainbow_img = np.zeros((grey_img.shape[0], grey_img.shape[1], 3))
    rainbow_img[:, :, 0] = np.clip(4*grey_img - 2, 0, 1.0) * (grey_img > 0.01) * (grey_img <= 1.0)
    rainbow_img[:, :, 1] =  np.clip(4*grey_img * (grey_img <=0.75), 0,1) + np.clip((-4*grey_img + 4) * (grey_img > 0.75), 0, 1)
    rainbow_img[:, :, 2] = np.clip(-4*grey_img + 2, 0, 1.0) * (grey_img > 0.01) * (grey_img <= 1.0)
    return rainbow_img

def sigmoid_rainbow_bsb_window(img):
    brain_img = sigmoid_window(img, 40, 80)
    subdural_img = sigmoid_window(img, 80, 200)
    bone_img = sigmoid_window(img, 600, 2000)
    combo = (brain_img*0.35 + subdural_img*0.5 + bone_img*0.15)
    combo_norm = (combo - np.min(combo)) / (np.max(combo) - np.min(combo))
    return map_to_gradient_sig(combo_norm)

view_images([sigmoid_rainbow_bsb_window(img) for img in imgs])

I haven't done enough experimenting to know which one of these works best. (Although I have a hunch it's *Sigmoid (Brain + Subdural + Bone) Windowing*.) 

I was thinking these might help add some diversity to an ensemble?

Thanks for reading and let me know if you have any questions or comments! **Also, if you found any of this interesting, don't forget to upvote.** 😄

## Acknowledgements <a></a>
Thanks again to [David Tang](https://www.kaggle.com/dcstang/see-like-a-radiologist-with-systematic-windowing), [Marco](https://www.kaggle.com/marcovasquez/basic-eda-data-visualization), [Nanashi](https://www.kaggle.com/jesucristo/rsna-introduction-eda-models), and [Richard McKinley](https://www.kaggle.com/omission/eda-view-dicom-images-with-correct-windowing) for sharing their excellent kernels.

Ideas were also borrowed from [Practical Window Setting Optimization for Medical Image Deep Learning](https://arxiv.org/pdf/1812.00572.pdf) and [Precise diagnosis of intracranial hemorrhage and subtypes using a three-dimensional joint convolutional and recurrent neural network](https://rd.springer.com/content/pdf/10.1007%2Fs00330-019-06163-2.pdf)

**Update 10/22: ** The `sigmoid_window` function can be pretty slow when processing the whole dataset on a cpu. You can get a nice speedup by running it on your gpu using cupy:

In [ ]:
import cupy as cp

def sigmoid_window(dcm, img, window_center, window_width, U=1.0, eps=(1.0 / 255.0)):
    img = cp.array(np.array(img))
    _, _, intercept, slope = get_windowing(dcm)
    img = img * slope + intercept
    ue = cp.log((U / eps) - 1.0)
    W = (2 / window_width) * ue
    b = ((-2 * window_center) / window_width) * ue
    z = W * img + b
    img = U / (1 + cp.power(np.e, -1.0 * z))
    img = (img - cp.min(img)) / (cp.max(img) - cp.min(img))
    return cp.asnumpy(img)